concat all csvs into 1 

dependencies

In [ ]:
pip install plotly
pip install datetime

In [77]:
import os
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
from datetime import datetime as dd


date_parse:
    input: str that represents the timestamp\
    returns: str that is a date

In [69]:
def date_parse(date_string):
    if len(date_string)>= 18:
        date_string = date_string[0:10]
        date_string = dd.strptime(date_string, '%Y-%m-%d').date()
    else:
        date_string = date_string.split(" ")[0]
        try:
            date_string = dd.strptime(date_string,'%m/%d/%Y').date()
        except:
            date_string = dd.strptime(date_string,'%m/%d/%y').date()
    return date_string

csv_concat:\
input: str that represents the folder\
returns: df that is a concat of all csvs in that folder


In [70]:
def csv_concat(folder):
    path = '/home/jovyan/work/github/covid_dashboard/' + folder + '/'
    files = os.listdir('/home/jovyan/work/github/covid_dashboard/' + folder)
    df_list = []
    for csv in files:
        df = pd.read_csv(path + csv)
        df_list.append(df)
    final = pd.concat(df_list, sort=False)[['Country/Region',
                                            'Confirmed',
                                            'Deaths',
                                            'Recovered', 
                                            'Last Update']]
    final['Last Update'] = final['Last Update'].apply(lambda x : date_parse(x))
    
    final = final.rename(columns = {'Country/Region':'country',
                                    'Confirmed':'confirmed',
                                    'Deaths':'deaths',
                                    'Recovered':'recovered', 
                                    'Last Update':'last_updated'})
    return final

In [73]:
print(csv_concat('data'))

           country  confirmed  deaths  recovered last_updated
0   Mainland China    65187.0  2615.0    20969.0   2020-02-26
1   Mainland China     1347.0     7.0      851.0   2020-02-26
2   Mainland China     1271.0    19.0     1033.0   2020-02-26
3      South Korea     1261.0    12.0       22.0   2020-02-26
4   Mainland China     1205.0     1.0      867.0   2020-02-26
..             ...        ...     ...        ...          ...
62              US        1.0     0.0        0.0   2020-02-01
63              US        1.0     0.0        0.0   2020-02-01
64              US        1.0     0.0        0.0   2020-02-01
65              US        1.0     0.0        0.0   2020-02-01
66              US        1.0     0.0        0.0   2020-02-01

[3767 rows x 5 columns]


In [102]:
data = csv_concat('data')
data = data.groupby(['last_updated']).agg({'confirmed':['sum'],
                                                      'deaths':['sum'],
                                                      'recovered':['sum']}).reset_index()
data.columns = ['last_updated','confirmed','deaths','recovered']
data

,last_updated,confirmed,deaths,recovered
0,2020-01-22,555.0,17.0,28.0
1,2020-01-23,653.0,18.0,30.0
2,2020-01-24,941.0,26.0,36.0
3,2020-01-25,1438.0,42.0,39.0
4,2020-01-26,2118.0,56.0,52.0
...,...,...,...,...
40,2020-03-02,91332.0,3091.0,46201.0
41,2020-03-03,94672.0,3176.0,48704.0
42,2020-03-04,93919.0,3248.0,50728.0
43,2020-03-05,96640.0,3336.0,53454.0


In [114]:
fig = px.line(data, x='last_updated', y='confirmed')
fig.add_scatter(x=data['last_updated'], y=data['deaths'])
fig.add_scatter(x=data['last_updated'], y=data['recovered'])
fig.update_layout(
    title = 'Covid-19 Tracker',
    xaxis_title = 'Day',
    yaxis_title = 'People')

fig.show()
# Not what is desired - need a line 